In [23]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import json
from PIL import Image 
import pprint
import os


In [24]:
train_data_dirs = []
train_data_map = "../../train.json"
val_data_dir = "../../val_imgs"
val_data_map = "../../val.json"

In [25]:
with open(train_data_map, 'r') as f:
    train_data = json.load(f)
with open(val_data_map, 'r') as f:
    val_data = json.load(f)

In [26]:
train_df = pd.DataFrame(train_data)
print("shape of data",train_df.shape)
print("count of missiing values\n",train_df.isna().sum())
train_df.head()

shape of data (4099419, 2)
count of missiing values
 caption    0
product    0
dtype: int64


,caption,product
0,Z-Couple Shook His Head Ornaments Doll Car Dec...,O1CN01ogOt7d1DHyjGtaaWT_!!192-0-lubanu.jpg
1,Z Remote Anti-Theft Door Wireless Door Anti-Th...,O1CN01eFgq841CxMtYCpqWF_!!147-0-lubanu.jpg
2,Z Qinghai qing liang Xidan Tip outside Microme...,O1CN01fXCPeA1SzgmI1mVLf_!!2207662892318.jpg
3,Z Magic Incognito Clothes Stack Pants Multi-La...,O1CN01qgixtq1BtPqrInOuX_!!3-0-lubanu.jpg
4,z ge zi yong pin yong ju ge zi zhuang men tie ...,TB1ZqfODEY1gK0jSZFCq6AwqXXa


In [27]:
all_files_df = []
sum_all_img = 0
for i in range(9):
    file_df = pd.DataFrame()
    folder = f"../../train_text_img_pairs_{i}/train_text_img_pairs_{i}_compressed/"
    file_df["product"] = os.listdir(folder)
    file_df["image_file_path"] = folder
    print(file_df.shape)
    sum_all_img += file_df.shape[0]
    all_files_df.append(file_df)

all_files_df= pd.concat(all_files_df)

(486971, 2)
(483293, 2)
(480947, 2)
(413462, 2)
(415712, 2)
(485469, 2)
(485522, 2)
(424338, 2)
(423219, 2)


In [28]:
all_files_df.head()
print(all_files_df.shape)

(4098933, 2)


In [33]:
image_df = pd.merge(train_df,all_files_df,on="product",how="inner")

In [35]:
image_df.shape

(4100105, 3)

caption            64944
product            64944
image_file_path    64944
dtype: int64

In [37]:
image_df.isna().sum()

caption            0
product            0
image_file_path    0
dtype: int64

In [34]:
val_df = pd.DataFrame(val_data)
print("shape of data",val_df.shape)
print("count of missiing values\n",val_df.isna().sum())
val_df.head()

shape of data (50000, 2)
count of missiing values
 caption    0
product    0
dtype: int64


,caption,product
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg


In [8]:
print("number of unique labels in train data",len(train_df["caption"].unique()))

number of unique labels in train data 3902657


In [9]:
print("number of unique labels in validiation data",len(val_df["caption"].unique()))

number of unique labels in validiation data 49715


In [10]:
train_df["caption"].value_counts()

Ghost mie zhi Edge Perimeter mercari Purchasing Direct Mail                                                                     28
Magic Fairy War Surrounding Stickers mercari Purchasing Direct Mail                                                             23
Off-the-Shelf Japan Imported Second-Hand Inflatable Lighter Have Mars Non-Arcing to Repair Sell Nothing in Return               19
Solid Wood Children with Guardrail Little Baby Boys and Girls Princess Bed Single Side Bed Widening Splicing Queen-Size Beds    19
Hook Strong zhan jiao tie Wall Wall-Mounted Load-Bearing Chuck Kitchen Hook Incognito Paste Door Free Punch Hook                17
                                                                                                                                ..
Tea Pot Cup Glass Filter Set Household Canister Purple Tank Tea Boiled Tea Leaf Gansu Tianshui Electric Furnace.                 1
Tea Retro Blue Glaze Hand-Painted ju ban Master Cup Kung Fu Tea Mug Tea ya shou bei

In [15]:
train_df[train_df["product"].str.slice(-3).isin(["jpeg"])]

,caption,product
